In [47]:
#import libraries
import pandas as pd
import numpy as np

<h1> Orders Export Preparation </h1>

Importing and cleaning orders data

In [48]:
#import dataframe of offers used from Jan 2019 - May 4 2020
orders_df = pd.read_csv('orders 2019_01_to_2020_05_04.csv')

#extract outlet_id, phone only
orders_df = orders_df[['outlet_id','phone','outlet_name']]

#remove NULLS
orders_df.count()
orders_df.dropna(subset=['phone', 'outlet_id'], inplace=True)
orders_df.reset_index(drop=True, inplace = True)
#the df should have outlet_ids and phone numbers only at this point, make sure data types are consistent
orders_df['phone'] = orders_df['phone'].astype(int)


<h1>Outlets Export Preparation</h1>

Calculate outlets' distance from target outlet

In [49]:
#select target outlet to use and input longitude latitude as variables
target_lat = 25.28744
target_long =  51.540345


In [50]:
#import outlets data outlet_id, outlet_name,lat, long columns
outlets_df = pd.read_csv('Outlets_2020_05_04.csv', header = 0,
                         encoding= 'unicode_escape')
outlets_df = outlets_df[['outlet_id','outlet_name','lat','long']]
outlets_df.head()

,outlet_id,outlet_name,lat,long
0,880,Alghais Diving Center - Banana Island Resort,25.296362,51.645839
1,2074,Koshari Station - Al Wakrah,25.179781,51.611892
2,243,Barahat Al Wakrah Restaurant,25.172543,51.611340
3,1013,Royal Tandoor - Souq Al Wakrah,25.168986,51.610198
4,1879,Smaisma Cafe - Wakrah,25.170867,51.609648


<h1>Calculating Distances from the Target Outlet </h1>

In [51]:
#calculate distances of the outlets from target outlet, add column distance from target (eudlidean) in km
#define haversine
from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

#finally, calculate
outlets_df['dist_km'] = [haversine(target_long,target_lat,outlets_df.long[i],outlets_df.lat[i]) for i in range(len(outlets_df))]



<h1>Filtering numbers who ordered within 5 km of the target outlet</h1>

In [52]:
#we use 6 km due to differences road distances (std of 3) output eligible for delivery, if not eligible for delivery
delivery_elig = outlets_df[outlets_df['dist_km'] <= 5]
delivery_elig = delivery_elig[['outlet_id','outlet_name','dist_km','lat','long']]
delivery_elig




,outlet_id,outlet_name,dist_km,lat,long
42,1706,Red Salon For Ladies,4.415014,25.253746,51.563573
43,418,Titanium Tint - Old Airport,4.341307,25.254208,51.563006
44,601,Baskin Robbins - Old Airport,4.320248,25.253901,51.562034
45,643,Dunkin Donuts - Old Airport,4.320248,25.253901,51.562034
46,1895,Bun Al Ameed - Old Airport,4.301717,25.254074,51.561996
47,567,Baskin Robbins - FFC Airport,3.298865,25.262110,51.557425
48,1005,Bombay Chowpatty - Old Airport,3.277518,25.262216,51.557209
49,628,Dunkin Donuts - FFC Airport,3.285412,25.262061,51.557076
50,1021,Tandoor - FFC,3.299597,25.261911,51.557073
51,714,Bodylines at Oryx Rotana Hotel,2.551827,25.268036,51.553896


In [53]:
#map data
import folium
latitude = 25.2854
longitude = 51.5310
eligible = folium.Map(location=[latitude, longitude], zoom_start=11) 

for lat, long, dist, label in zip(delivery_elig['lat'], delivery_elig['long'], delivery_elig['dist_km'], delivery_elig['outlet_name']):
    label = folium.Popup(label, parse_html=True)
    if dist <0.00001:
        color = 'red'
    else:
        color = 'green'
    folium.features.CircleMarker(
        [lat, long],
        radius=5,
        color= color,
        fill = True,
        popup = label,
        fill_color='green',
    ).add_to(eligible)
    

# display map

eligible

In [54]:
#merge with orders export to find phone numbers who ordered from eligible outlets, Nan under dist_km means not <10km
target_user = orders_df.merge(delivery_elig[['outlet_id','dist_km']],on="outlet_id", how = 'left')

#select only numbers that are eligible and remove nan from dist_km
target_user = target_user.dropna(subset=['dist_km'])
target_user

,outlet_id,phone,outlet_name,dist_km
5,985,97466952011,Papa John's - Old Airport Area,4.258450
7,175,97430219199,Papa John's - Emadi,0.819486
8,974,97450561620,Papa John's - Salwa Road,4.767178
9,988,97430881911,Papa John's - Umm Salal Ali,0.819151
10,981,97433254876,Papa John's - Ezdan Tower,3.971105
11,974,97433666219,Papa John's - Salwa Road,4.767178
12,955,97455117381,The Coffee Bean & Tea Leaf - Kidney Center,4.489575
13,2144,97455104880,Maison de Sushi - Tornado Tower,3.725356
15,2144,97455104880,Maison de Sushi - Tornado Tower,3.725356
17,974,97455807360,Papa John's - Salwa Road,4.767178


<h1>Final List Filtering: Select ONLY OOREDOO and REMOVE DUPLICATES</h1>

In [55]:
#import registration export
registration = pd.read_csv('registered_customers 2020.05.04.csv')
registration = registration[['phone','network']]

#merge target user data frame with registered export to find ooredoo numbers
target_user = target_user.merge(registration,on="phone",copy = False)

#select only numbers that are ooredoo
target_user = target_user[target_user['network'] == 'ooredoo']

target_user

,outlet_id,phone,outlet_name,dist_km,network
0,985,97466952011,Papa John's - Old Airport Area,4.258450,ooredoo
1,1922,97466952011,Lamazani Grill - Najma,2.358547,ooredoo
2,175,97430219199,Papa John's - Emadi,0.819486,ooredoo
3,985,97430219199,Papa John's - Old Airport Area,4.258450,ooredoo
4,998,97430219199,Shawarma Station - Al Hilal,3.283323,ooredoo
5,998,97430219199,Shawarma Station - Al Hilal,3.283323,ooredoo
6,985,97430219199,Papa John's - Old Airport Area,4.258450,ooredoo
7,974,97450561620,Papa John's - Salwa Road,4.767178,ooredoo
8,954,97450561620,The Coffee Bean & Tea Leaf - Al Mana,3.399316,ooredoo
9,954,97450561620,The Coffee Bean & Tea Leaf - Al Mana,3.399316,ooredoo


In [56]:
#remove duplicates and select phone numbers only
target_user.drop_duplicates(subset ="phone", keep = False, inplace = True)
target_user = target_user[['phone']]
#save to csv
target_user.to_csv(r'Targeting Maxill.csv', index = False)
target_user

,phone
71,97455117381
102,97450707131
138,97433522013
157,97450016128
158,97455282944
169,97466331039
200,97470564470
201,97455463449
251,97433833412
266,97455389008


Note: data will miss order rows from outlets without location data.